https://itsembedded.com/dhd/vivado_sim_1/ Vivado Simulator scripted flow Part 1: Basic CLI usage
Using vivado.


Lean to verilog? Lean as a cocotb callback?
Glue via python coctb?

See also
- 2025/3 Hardware
- verilog_td4


https://github.com/mb-sat/ulx3s-longwave-sdr FPGA Stream Software Defined Radio https://codeberg.org/Mecrisp/ulx3s-longwave-sdr





https://www2.imm.dtu.dk/pubdb/edoc/imm855.pdf Asynchronous Circuit Design A Tutorial. Jens Sparsø
https://dl.acm.org/doi/pdf/10.1145/63526.63532  MICROPIPELINES - IVAN E. SUTHERLAND 

Analog stuff in circuits.ipynb verilog-a 

https://journal.hep.com.cn/fcs/EN/10.1007/s11704-024-40127-0  PyABV: a framework for enhancing PyRTL with assertion-based verification

https://kastner.ucsd.edu/hlsbook/ Parallel Programming for FPGAs
https://pp4fpgas.readthedocs.io/en/latest/ labs

HLS

HDL tookit matlab


Call verilator as a single shot? We could call simulator with input vector and parse output vector. This is kind of what Silviu was saying. Just print your output and compare. For analyzing a hypothetical sound processing or video, this is much easier.

Discrete events systems. https://en.wikipedia.org/wiki/Discrete-event_dynamic_system  https://en.wikipedia.org/wiki/Discrete-event_simulation Simulators systems 

https://www.cs.princeton.edu/~ad4048/pdfs/formal-verification-of-hardware-using-mlir.pdf

# Cocotb

UVM pyuvm https://github.com/pyuvm/pyuvm
https://arxiv.org/abs/2407.10317  Towards Efficient Design Verification -- Constrained Random Verification using PyUVM
cocotb. How does it work. Triggers.
Cocotb but lean

Lean cocotb? ffi out of pytho nto lean or directly make verilog libraries from lean code.

Gotta get oss-cad off my path. I dunno where it is though.

Ugh. I guess the makefile is the way to go. 3 files, verilog, python bench code and makefile? Sickening. I mean it makes sense. There has to be code running in inner interpreter, it needs rtl, and you gotta run it. I guess there could be 2 files + a command line program.


In [1]:
%%file /tmp/halfadd.v
module halfadd(
    input a,
    input b,
    output sum,
    output carry
);
    assign sum = a ^ b;
    assign carry = a & b;
endmodule


Writing /tmp/halfadd.v


In [33]:
%%file /tmp/test_my_design.py

import cocotb
from cocotb.triggers import FallingEdge, Timer


async def generate_clock(dut):
    """Generate clock pulses."""

    for cycle in range(10):
        dut.clk.value = 0
        await Timer(1, units="ns")
        dut.clk.value = 1
        await Timer(1, units="ns")


@cocotb.test()
async def my_second_test(dut):
    """Try accessing the design."""

    await cocotb.start(generate_clock(dut))  # run the clock "in the background"

    await Timer(5, units="ns")  # wait a bit
    await FallingEdge(dut.clk)  # wait for falling edge/"negedge"

    dut._log.info("my_signal_1 is %s", dut.my_signal_1.value)
    assert dut.my_signal_2.value[0] == 0, "my_signal_2[0] is not 0!"

Writing /tmp/test_my_design.py


In [ ]:
%%file /tmp/Makefile
SIM ?= icarus
TOPLEVEL_LANG ?= verilog
VERILOG_SOURCES += $(PWD)/halfadd.v
TOPLEVEL = halfadd
MODULE = test_my_design
include $(shell cocotb-config --makefiles)/Makefile.sim

Overwriting /tmp/Makefile


In [34]:
! cd /tmp && make

rm -f results.xml
"make" -f Makefile results.xml
make[1]: Entering directory '/tmp'
rm -f results.xml
MODULE=test_my_design TESTCASE= TOPLEVEL=halfadd TOPLEVEL_LANG=verilog \
         /usr/bin/vvp -M /home/philip/philzook58.github.io/.venv/lib/python3.12/site-packages/cocotb/libs -m libcocotbvpi_icarus   sim_build/sim.vvp  
     -.--ns INFO     gpi                                ..mbed/gpi_embed.cpp:108  in set_program_name_in_venv        Using Python virtual environment interpreter at /home/philip/philzook58.github.io/.venv/bin/python
     -.--ns INFO     gpi                                ../gpi/GpiCommon.cpp:101  in gpi_print_registered_impl       VPI registered
     0.00ns INFO     cocotb                             Running on Icarus Verilog version 12.0 (stable)
     0.00ns INFO     cocotb                             Running tests with cocotb v1.9.2 from /home/philip/philzook58.github.io/.venv/lib/python3.12/site-packages/cocotb
     0.00ns INFO     cocotb                         

In [52]:
%%file /tmp/mytest.py
import cocotb
from cocotb.clock import Clock
from cocotb.triggers import RisingEdge
from cocotb.runner import get_runner
#from cocotb_tools.runner import get_runner

@cocotb.test()
async def dff_simple_test(dut):
    """Test that d propagates to q"""

    print(dut)
    print("HHEHEEYEY")
    dut._log.info("my_signal_1 is hohohoo")
    assert False, "hey there"

def test_simple_dff_runner():
    runner = get_runner("icarus")
    runner.build(
        verilog_sources=["/tmp/halfadd.v"],
        hdl_toplevel="halfadd",
        always=True,
    )

    runner.test(hdl_toplevel="halfadd", test_module="mytest",testcase="dff_simple_test")

if __name__ == "__main__":
    test_simple_dff_runner()

Overwriting /tmp/mytest.py


In [53]:
! cd /tmp && python /tmp/mytest.py

/tmp/mytest.py:4: UserWarning: Python runners and associated APIs are an experimental feature and subject to change.
  from cocotb.runner import get_runner
INFO: Running command iverilog -o /tmp/sim_build/sim.vvp -D COCOTB_SIM=1 -s halfadd -g2012 /tmp/halfadd.v in directory /tmp/sim_build
INFO: Running command vvp -M /home/philip/philzook58.github.io/.venv/lib/python3.12/site-packages/cocotb/libs -m libcocotbvpi_icarus /tmp/sim_build/sim.vvp in directory /tmp/sim_build
     -.--ns INFO     gpi                                ..mbed/gpi_embed.cpp:108  in set_program_name_in_venv        Using Python virtual environment interpreter at /home/philip/philzook58.github.io/.venv/bin/python
     -.--ns INFO     gpi                                ../gpi/GpiCommon.cpp:101  in gpi_print_registered_impl       VPI registered
     0.00ns INFO     cocotb                             Running on Icarus Verilog version 12.0 (stable)
     0.00ns INFO     cocotb                             Running tests with

# Alt Verilog Binding

pyverilator
https://github.com/csail-csg/pyverilator
https://github.com/bat52/pyverilator

https://github.com/namin/bluespec-sandbox hmm.
https://github.com/mit-plv/koika/blob/master/examples/rv/etc/rvcore.pyverilator.py hmmmm

# Bluespec

enable and ready wires. Similar to network / bus?


Bluespec  https://www.cs.uoregon.edu/research/summerschool/summer18/lectures/L3-BSV_ConcurrencyAndSemantics.pdf as rewrite rules? https://types.pl/@sandmouth/114937308721349573

So Augustsson worked on bluepsec at some point?

https://github.com/B-Lang-org/bsc compiled release

https://github.com/rsnikhil/Bluespec_BSV_Tutorial
https://github.com/rsnikhil/ICFP2020_Bluespec_Tutorial/blob/master/ICFP2020_Bluespec_Tutorial.adoc

https://dl.acm.org/doi/10.1145/3385412.3385965  The essence of Bluespec: a core language for rule-based hardware design. Koika

https://cas.ee.ic.ac.uk/people/ssingh/bluespec_l3l4.pdf

https://yehowshuaimmanuel.com/posts/getting-started-with-bluespec/

https://www.youtube.com/watch?v=ff0eDTN8ckM  OPLSS'24: Thomas Bourgeat https://www.cs.uoregon.edu/research/summerschool/summer24/lectures/Bourgeat1.pdf

clash
rachit stuff
spinal chisel


In [1]:
! /home/philip/Downloads/bsc-2025.01.1-ubuntu-24.04/bin/bsc --help

Usage:
  bsc -help                                to get help
  bsc [flags] file.bsv                     to partially compile a Bluespec file
  bsc [flags] -verilog -g mod file.bsv     to compile a module to Verilog
  bsc [flags] -verilog -g mod -u file.bsv  to recursively compile modules to Verilog
  bsc [flags] -verilog -e topmodule        to link Verilog into a simulation model
  bsc [flags] -sim -g mod file.bsv         to compile to a Bluesim object
  bsc [flags] -sim -g mod -u file.bsv      to recursively compile to Bluesim objects
  bsc [flags] -sim -e topmodule            to link objects into a Bluesim binary
  bsc [flags] -systemc -e topmodule        to link objects into a SystemC model

Compiler flags:
-D macro                define a macro for the BSV or Verilog preprocessor
-E                      run just the preprocessor, dumping result to stdout
-I path                 include path for compiling foreign C/C++ source
-L path                 library path for linking foreign

In [2]:
%%file /tmp/hello.bs
mkTop :: Module Empty
mkTop =
  module
    rules
      "rl_print_answer": when True ==> do
          $display "\n\n***** Deep Thought says: Hello, World! *****"
          $display "      And the answer is: %0d (or, in hex: 0x%0h)\n"  42  42
          $finish


Writing /tmp/hello.bs


In [4]:
! /home/philip/Downloads/bsc-2025.01.1-ubuntu-24.04/bin/bsc -sim -g mkTop /tmp/hello.bs 

Error: "/tmp/hello.bs", line 1, column 0: (P0005)
  Unexpected "mkTop"; expected "package"


# Simulators

hwil

systemc 
https://en.wikipedia.org/wiki/SystemC
https://github.com/denisgav/v2sc
verilator exports systemc?
https://www.cprover.org/hardware/v2c/

gen5
simics
renode

cache coherence
determinsim cycle accuracy




# Logic Syntehsis
https://en.wikipedia.org/wiki/Logic_synthesis

https://link.springer.com/book/10.1007/b117060 
Logic Synthesis and Verification Algorithms

Hmm. If I could get an fpga simulator or circuit simulator working, that could be a virtuous cycle.
https://www.coursera.org/learn/vlsi-cad-logic 
https://www.youtube.com/playlist?list=PLUGkvWkzb6cUkOqCfMIsbMzdSLl9CBLRy
